In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
reports_dir = 'reports'
def get_file_path(measurement, file_name):
    return os.path.join(reports_dir, measurement, file_name)

In [121]:
def parse_name(name):
    args_list = name.split('-')
    args = {}
    args['Element Type'] = args_list[0]
    try:
        args['Interpolation Type'] = args_list[1]
    except:
        print(name)
    if args_list[2][0] == '0':
        args['Element Size'] = float('.'+args_list[2][1:])
    else:
        args['Element Size'] = float(args_list[2])
    args['Measurement'] = args_list[3][:-4]
    if args['Measurement'] == 'sm':
        measurement = 'S, Mises'
    elif args['Measurement'] == 'u':
        measurement = 'U, Magnitude'
    else:
        raise ValueError("Unknown measurement type.")
    args['Measurement Type'] = measurement
    return args

In [56]:

def read_coordinates_from_file(file_name):

    # Open the file and read all lines
    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Initialize variables for searching the start of the coordinates table
    start_line = None
    header_found = False
    
    # Search for the header line that includes 'Node ID' and 'Orig. Coords'
    for i in range(len(lines)):
        if "Part Instance" in lines[i] and "Node ID" in lines[i] and "Orig. Coords" in lines[i]:
            start_line = i
            header_found = True
            break

    if not header_found:
        raise ValueError("Coordinates table not found in the file.")
    
    # Extract the header and the table data lines
    table_lines = lines[start_line+1:]

    # Initialize lists to store table data
    part_instances = []
    node_ids = []
    orig_coords_x = []
    orig_coords_y = []
    orig_coords_z = []

    # Loop over the table lines and parse the data
    for line in table_lines:
        columns = line.strip().split()

        # Ensure the line has at least 6 columns (Part Instance, Node ID, X, Y, Z)
        # and the third column is not a header or other text (which would raise the ValueError)
        if len(columns) >= 6 and columns[2].replace('.', '', 1).replace('-', '', 1).isdigit():
            part_instances.append(columns[0])
            node_ids.append(columns[1])
            orig_coords_x.append(float(columns[2]))
            orig_coords_y.append(float(columns[3]))
            orig_coords_z.append(float(columns[4]))

    # Create a DataFrame from the collected data
    data = {
        'Node ID': node_ids,
        'Orig X': orig_coords_x,
        'Orig Y': orig_coords_y,
        'Orig Z': orig_coords_z,
    }

    df = pd.DataFrame(data)
    
    return df



In [57]:
nodes_dict = pd.DataFrame({
    'Node ID': {0: 'A', 1: 'B', 2: 'C', 3: 'D'}, 
    'Orig X': {0: 25.2539, 1: 28.9005, 2: 14.2395, 3: -3.61857}, 
    'Orig Y': {0: -15.0587, 1: 7.19905, 2: 43.8071, 3: -24.8145}, 
    'Orig Z': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
    })
nodes_dict


,Node ID,Orig X,Orig Y,Orig Z
0,A,25.25390,-15.05870,0.0
1,B,28.90050,7.19905,0.0
2,C,14.23950,43.80710,0.0
3,D,-3.61857,-24.81450,0.0


In [59]:
def translate_nodes_ID(file_name):
    nodes_from_mesh = read_coordinates_from_file(file_name)
    merged_df = pd.merge(nodes_from_mesh, nodes_dict, on=['Orig X', 'Orig Y', 'Orig Z'], suffixes=('_var', '_const'))
    # Create a dictionary that maps the variable Node ID to the constant Node ID
    translation_dict = dict(zip(merged_df['Node ID_var'], merged_df['Node ID_const']))
    return translation_dict

In [62]:
def convert_df_node_IDs(df, translation_dict):
    # Replace the variable Node IDs with the constant Node IDs
    df['Node ID'] = df['Node ID'].map(translation_dict)
    return df

In [199]:
def get_no_nodes(job_file_path):
    with open(job_file_path, 'r') as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if '*Nset' in line:
            break
    no_nodes_line = lines[i-1].split(',')
    print(no_nodes_line)
    no_nodes = int(no_nodes_line[0])
    return no_nodes


In [200]:
def job_file_name(rpt_file_name):
    file_name = '-'.join(rpt_file_name.split('-')[:-1]) + '.inp'
    file_path = os.path.join('..','extra-files', 'job-inp-files', file_name)
    return file_path

In [201]:
os.getcwd()
os.chdir('reports')

In [210]:

def read_table_from_file(file_name):

    params = parse_name(file_name)

    
    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Start from the bottom of the file to find where the table begins
    start_line = None
    header_found = False
    for i in range(len(lines)-1, -1, -1):
        if "Part Instance" in lines[i] and "Node ID" in lines[i]:
            start_line = i
            header_found = True
            break

    if not header_found:
        raise ValueError("Table start not found in the file.")
    
    # Extract the header and the table data lines
    header_line = lines[start_line].strip().split()
    table_lines = lines[start_line+1:]

    # Initialize lists to store table data
    part_instances = []
    node_ids = []
    attached_elements = []
    measurements = []

    # Loop over the table lines and parse the data
    for line in table_lines:
        # Assuming the data is space-separated (or adjust if there's a different delimiter)
        columns = line.strip().split()

        # Only process lines with expected number of columns (adjust based on actual table structure)
        if len(columns) >= 4:
            part_instances.append(columns[0])
            node_ids.append(columns[1])
            attached_elements.append(columns[2])
            measurements.append(float(columns[3]))


    no_elements = get_no_nodes(job_file_name(file_name))
    
    translation_dict = translate_nodes_ID(file_name)

    # Create a DataFrame from the collected data
    data = {
        'Part Instance': part_instances,
        'Node ID': node_ids,
        'Attached elements': attached_elements,
        'Values': measurements,
        'Measurement Type': np.full(len(part_instances), params['Measurement Type']),
        'Element Type': np.full(len(part_instances), params['Element Type']),
        'Interpolation Type': np.full(len(part_instances), params['Interpolation Type']),
        'Element Size': np.full(len(part_instances), params['Element Size']),
        'Number of Elements': np.full(len(part_instances), no_elements)
    }

    df = pd.DataFrame(data)

    df = convert_df_node_IDs(df, translation_dict)
    
    return df

# Example usage
file = 'quad-quad-025-sm.rpt'  # Replace with the path to your file
df = read_table_from_file(file)

# Display the DataFrame
df


['64144', '  43467', '  43453', '  43337', '  43511', ' 184411', ' 184178', ' 184103', '  77295\n']


,Part Instance,Node ID,Attached elements,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements
0,OSSO-4,A,1278,0.215003,"S, Mises",quad,quad,0.25,64144
1,OSSO-4,A,1327,0.215003,"S, Mises",quad,quad,0.25,64144
2,OSSO-4,B,1119,0.178939,"S, Mises",quad,quad,0.25,64144
3,OSSO-4,B,20920,0.178939,"S, Mises",quad,quad,0.25,64144
4,OSSO-4,C,1962,0.105825,"S, Mises",quad,quad,0.25,64144
5,OSSO-4,C,2094,0.105825,"S, Mises",quad,quad,0.25,64144
6,OSSO-4,D,4512,0.089411,"S, Mises",quad,quad,0.25,64144
7,OSSO-4,D,21183,0.089411,"S, Mises",quad,quad,0.25,64144


In [211]:
dfs = []
error_counter = 0
for i, file in enumerate(os.listdir()):
    try:
        df = read_table_from_file(file)
        dfs.append(df)
        print(f"[[[[[               {i}               ]]]]]")
        print(df.head())
    except:
        error_counter += 1
        print(f"Failed to read {file}") 

['70', ' 43', ' 46', ' 64', ' 63\n']
[[[[[               0               ]]]]]
  Part Instance Node ID Attached elements    Values Measurement Type  \
0        OSSO-4       A                22  0.291779         S, Mises   
1        OSSO-4       A                38  0.291779         S, Mises   
2        OSSO-4       B                52  0.121246         S, Mises   
3        OSSO-4       B                58  0.121246         S, Mises   
4        OSSO-4       C                10  0.152956         S, Mises   

  Element Type Interpolation Type  Element Size  Number of Elements  
0         quad                lin          10.0                  70  
1         quad                lin          10.0                  70  
2         quad                lin          10.0                  70  
3         quad                lin          10.0                  70  
4         quad                lin          10.0                  70  
['6780', '  3512', '  2223', '   721', ' 13450', ' 11306', ' 13451\n

In [195]:
#os.chdir('reports')
os.chdir('..')

In [106]:
sm = os.listdir()
lin = [file for file in sm if parse_name(file)['Interpolation Type'] == 'lin']
quad = [file for file in sm if parse_name(file)['Interpolation Type'] == 'quad']
lin_tri = [file for file in lin if parse_name(file)['Element Type'] == 'tri']
lin_quad = [file for file in lin if parse_name(file)['Element Type'] == 'quad']
quad_tri = [file for file in quad if parse_name(file)['Element Type'] == 'tri']
quad_quad = [file for file in quad if parse_name(file)['Element Type'] == 'quad']

desired = quad_quad

print(len(desired))
desired

6


['quad-quad-1-sm.rpt',
 'quad-quad-3-sm.rpt',
 'quad-quad-025-sm.rpt',
 'quad-quad-5-sm.rpt',
 'quad-quad-05-sm.rpt',
 'quad-quad-2-sm.rpt']

In [212]:
mega_df = pd.concat(dfs, axis=0, ignore_index=True)
mega_df

,Part Instance,Node ID,Attached elements,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements
0,OSSO-4,A,22,0.291779,"S, Mises",quad,lin,10.0,70
1,OSSO-4,A,38,0.291779,"S, Mises",quad,lin,10.0,70
2,OSSO-4,B,52,0.121246,"S, Mises",quad,lin,10.0,70
3,OSSO-4,B,58,0.121246,"S, Mises",quad,lin,10.0,70
4,OSSO-4,C,10,0.152956,"S, Mises",quad,lin,10.0,70
...,...,...,...,...,...,...,...,...,...
543,OSSO-4,B,441,0.153708,"S, Mises",quad,lin,2.0,1024
544,OSSO-4,C,128,0.097691,"S, Mises",quad,lin,2.0,1024
545,OSSO-4,C,377,0.097691,"S, Mises",quad,lin,2.0,1024
546,OSSO-4,D,209,0.095077,"S, Mises",quad,lin,2.0,1024


In [214]:
mega_df.to_csv('mega_df.csv', index=False)

In [198]:
mega_df

,Part Instance,Node ID,Attached elements,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements
0,OSSO-4,A,1567,0.002585,"U, Magnitude",tri,quad,1.0,13803
1,OSSO-4,A,1696,0.002585,"U, Magnitude",tri,quad,1.0,13803
2,OSSO-4,A,1741,0.002585,"U, Magnitude",tri,quad,1.0,13803
3,OSSO-4,B,1198,0.004218,"U, Magnitude",tri,quad,1.0,13803
4,OSSO-4,B,1579,0.004218,"U, Magnitude",tri,quad,1.0,13803
...,...,...,...,...,...,...,...,...,...
519,OSSO-4,B,441,0.153708,"S, Mises",quad,lin,2.0,1086
520,OSSO-4,C,128,0.097691,"S, Mises",quad,lin,2.0,1086
521,OSSO-4,C,377,0.097691,"S, Mises",quad,lin,2.0,1086
522,OSSO-4,D,209,0.095077,"S, Mises",quad,lin,2.0,1086


In [217]:
os.getcwd()
os.chdir('..')

In [219]:
new_df = pd.read_csv('mega_df.csv')
new_df.drop(columns=['Attached elements'], inplace=True)
new_df

,Part Instance,Node ID,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements
0,OSSO-4,A,0.291779,"S, Mises",quad,lin,10.0,70
1,OSSO-4,A,0.291779,"S, Mises",quad,lin,10.0,70
2,OSSO-4,B,0.121246,"S, Mises",quad,lin,10.0,70
3,OSSO-4,B,0.121246,"S, Mises",quad,lin,10.0,70
4,OSSO-4,C,0.152956,"S, Mises",quad,lin,10.0,70
...,...,...,...,...,...,...,...,...
543,OSSO-4,B,0.153708,"S, Mises",quad,lin,2.0,1024
544,OSSO-4,C,0.097691,"S, Mises",quad,lin,2.0,1024
545,OSSO-4,C,0.097691,"S, Mises",quad,lin,2.0,1024
546,OSSO-4,D,0.095077,"S, Mises",quad,lin,2.0,1024


In [221]:
new_df.drop_duplicates(inplace=True, keep='first')

In [229]:
new_df

,Part Instance,Node ID,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements
0,OSSO-4,A,0.291779,"S, Mises",quad,lin,10.0,70
2,OSSO-4,B,0.121246,"S, Mises",quad,lin,10.0,70
4,OSSO-4,C,0.152956,"S, Mises",quad,lin,10.0,70
6,OSSO-4,D,0.296456,"S, Mises",quad,lin,10.0,70
8,OSSO-4,A,0.002585,"U, Magnitude",tri,quad,1.0,6780
...,...,...,...,...,...,...,...,...
537,OSSO-4,D,0.092332,"S, Mises",tri,quad,2.0,1737
540,OSSO-4,A,0.223038,"S, Mises",quad,lin,2.0,1024
542,OSSO-4,B,0.153708,"S, Mises",quad,lin,2.0,1024
544,OSSO-4,C,0.097691,"S, Mises",quad,lin,2.0,1024


In [223]:
new_df.to_csv('mega_df2.csv', index=False)

In [231]:
import re
from datetime import datetime

# Define the function to extract the first and last timestamp and compute the difference
def calculate_time_difference(file):
    with open(f'extra-files/log-files/{file}', 'r') as file:
        file_content = file.read()
        # Use regular expressions to find all timestamps in the file
        timestamp_pattern = r'\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}'
        timestamps = re.findall(timestamp_pattern, file_content)
        
        # Ensure there are at least two timestamps
        if len(timestamps) >= 2:
            # Parse the timestamps into datetime objects
            time_format = "%d/%m/%Y %H:%M:%S"
            first_time = datetime.strptime(timestamps[0], time_format)
            last_time = datetime.strptime(timestamps[-1], time_format)
            
            # Calculate the difference
            time_difference = last_time - first_time
            return time_difference
        else:
            return None


In [232]:
for file in os.listdir('extra-files/log-files'):
    params_list = file.split('-')
    args = {}
    args['Element Type'] = params_list[0]
    try:
        args['Interpolation Type'] = params_list[1]
    except:
        print(file)
    if params_list[2][0] == '0':
        args['Element Size'] = float('.'+params_list[2][1:-4])
    else:
        args['Element Size'] = float(params_list[2][:-4])

    time_diff = calculate_time_difference(file)
    
    # Filter the rows in new_df that match the parameters in args
    mask = (
        (new_df['Element Type'] == args['Element Type']) &
        (new_df['Interpolation Type'] == args['Interpolation Type']) &
        (new_df['Element Size'] == args['Element Size'])
    )

    # Add the 'Time Taken' column and populate it with the time_diff for the matching rows
    new_df.loc[mask, 'Time Taken'] = time_diff

    

In [237]:
new_df.rename(columns={'Time Taken': 'Time Taken (s)'}, inplace=True)
new_df

,Part Instance,Node ID,Values,Measurement Type,Element Type,Interpolation Type,Element Size,Number of Elements,Time Taken (s)
0,OSSO-4,A,0.291779,"S, Mises",quad,lin,10.0,70,16.0
2,OSSO-4,B,0.121246,"S, Mises",quad,lin,10.0,70,16.0
4,OSSO-4,C,0.152956,"S, Mises",quad,lin,10.0,70,16.0
6,OSSO-4,D,0.296456,"S, Mises",quad,lin,10.0,70,16.0
8,OSSO-4,A,0.002585,"U, Magnitude",tri,quad,1.0,6780,16.0
...,...,...,...,...,...,...,...,...,...
537,OSSO-4,D,0.092332,"S, Mises",tri,quad,2.0,1737,17.0
540,OSSO-4,A,0.223038,"S, Mises",quad,lin,2.0,1024,17.0
542,OSSO-4,B,0.153708,"S, Mises",quad,lin,2.0,1024,17.0
544,OSSO-4,C,0.097691,"S, Mises",quad,lin,2.0,1024,17.0


In [238]:
new_df.to_csv('mega_df3.csv', index=False)